In [10]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [14]:
client.delete_experiment("4")

In [18]:
client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/mlops/mlruns/3', creation_time=1681273869996, experiment_id='3', last_update_time=1681273869996, lifecycle_stage='active', name='nyc-taxi-experiment-2', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops/mlruns/2', creation_time=1681187419970, experiment_id='2', last_update_time=1681187419970, lifecycle_stage='active', name='nyc-taxi-experiment-1', tags={}>]

In [27]:
client.search_runs(experiment_ids=2)[1]

<Run: data=<RunData: metrics={'rmse': 21.330564452067502,
 'training_mean_absolute_error': 1.2663162579498975e-13,
 'training_mean_squared_error': 3.132248556025532e-23,
 'training_r2_score': 1.0,
 'training_root_mean_squared_error': 5.5966494941398035e-12,
 'training_score': 1.0}, params={'bootstrap': 'False',
 'ccp_alpha': '0.0',
 'criterion': 'squared_error',
 'max_depth': 'None',
 'max_features': '1.0',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'n_estimators': '100',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': 'None',
 'train-data-path': './dataset/green_tripdata_2022-10.parquet',
 'val-data-path': './dataset/green_tripdata_2022-11.parquet',
 'verbose': '0',
 'warm_start': 'False'}, tags={'data_scientist': 'Soumyadip Majumder',
 'estimator_class': 'sklearn.ensemble._forest.ExtraTreesRegressor',
 'estimator_name': 'ExtraTreesRegressor',
 

In [23]:
runs = client.search_runs(
                    experiment_ids="2",
                    filter_string="metrics.rmse < 25",
                    run_view_type=ViewType.ACTIVE_ONLY,
                    max_results=5,
                    order_by=["metrics.rmse ASC"]
)

In [33]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']}:.4f")

run id: 443595f2244e486fb7bed5bd3f3001cd, rmse: 4.6840243946542605e-05:.4f
run id: b0d56310fbe24178a254c4d3fcfd4700, rmse: 21.330564452067502:.4f


In [41]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [43]:
run_id = "443595f2244e486fb7bed5bd3f3001cd"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/04/12 05:25:37 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 6
Created version '6' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1681277137803, current_stage='None', description=None, last_updated_timestamp=1681277137803, name='nyc-taxi-regressor', run_id='443595f2244e486fb7bed5bd3f3001cd', run_link=None, source='/home/ubuntu/mlops/mlruns/2/443595f2244e486fb7bed5bd3f3001cd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [44]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 6, stage: None
version: 4, stage: Staging


In [46]:
model_version = 4
new_stage = "Production"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

model_version = 6
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1681277137803, current_stage='Staging', description=None, last_updated_timestamp=1681277247429, name='nyc-taxi-regressor', run_id='443595f2244e486fb7bed5bd3f3001cd', run_link=None, source='/home/ubuntu/mlops/mlruns/2/443595f2244e486fb7bed5bd3f3001cd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [47]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1681277137803, current_stage='Staging', description='The model version 6 was transitioned to Staging on 2023-04-12', last_updated_timestamp=1681277869696, name='nyc-taxi-regressor', run_id='443595f2244e486fb7bed5bd3f3001cd', run_link=None, source='/home/ubuntu/mlops/mlruns/2/443595f2244e486fb7bed5bd3f3001cd/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [30]:
import mlflow
logged_model = 'runs:/443595f2244e486fb7bed5bd3f3001cd/models_nyc_dur_pred'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

MissingConfigException: Yaml file '/home/ubuntu/mlops/mlruns/2/443595f2244e486fb7bed5bd3f3001cd/meta.yaml' does not exist.

In [52]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}


In [53]:
df = read_dataframe("./dataset/green_tripdata_2022-11.parquet")

In [54]:
import pickle

with open("models/dv_nyc_taxi.bin", "rb") as f_in:
    dv = pickle.load(f_in)

In [55]:
X_test = preprocess(df, dv)
target = "duration"
y_test = df[target].values

In [56]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

CPU times: user 1.56 s, sys: 124 ms, total: 1.68 s
Wall time: 1.68 s


{'rmse': 3467.0034080630744}

In [57]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 30.7 ms, sys: 0 ns, total: 30.7 ms
Wall time: 29.7 ms


{'rmse': 3476.8452675803806}

In [58]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1681272066843, current_stage='Production', description='', last_updated_timestamp=1681278505411, name='nyc-taxi-regressor', run_id='b0d56310fbe24178a254c4d3fcfd4700', run_link='', source='/home/ubuntu/mlops/mlruns/2/b0d56310fbe24178a254c4d3fcfd4700/artifacts/models_nyc_dur_pred', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [59]:
model_name

'nyc-taxi-regressor'